## Upload image:

In [ ]:
from google.colab import files
import io
import os
from PIL import Image

uploaded = files.upload()

if uploaded:
    output_folder = "input"
    os.makedirs(output_folder, exist_ok=True)
    print(f"Saving uploaded images to: /{output_folder}/")

    for filename, content in uploaded.items():
        # Load the image using PIL
        image = Image.open(io.BytesIO(content))
        # Save the image to the specified output folder
        save_path = os.path.join(output_folder, filename)
        image.save(save_path)

    print(f"Images saved to {save_path}")
else:
    print("No files were uploaded.")

TypeError: 'NoneType' object is not subscriptable

## Inference for blood cells detection

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.9 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
model = YOLO('/content/best_part_1.pt')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os

input_images_folder = "./input"
results = model.predict(source=input_images_folder, save=True, conf=0.25, iou=0.7)
print("\nInference complete!")


image 1/12 /content/input/BloodImage_00004.jpg: 480x640 18 RBCs, 1 WBC, 359.7ms
image 2/12 /content/input/BloodImage_00005.jpg: 480x640 24 RBCs, 1 WBC, 3 Plateletss, 165.9ms
image 3/12 /content/input/BloodImage_00006.jpg: 480x640 21 RBCs, 1 WBC, 2 Plateletss, 155.0ms
image 4/12 /content/input/BloodImage_00007.jpg: 480x640 26 RBCs, 1 WBC, 1 Platelets, 194.9ms
image 5/12 /content/input/BloodImage_00008.jpg: 480x640 23 RBCs, 1 WBC, 151.5ms
image 6/12 /content/input/BloodImage_00009.jpg: 480x640 24 RBCs, 1 WBC, 2 Plateletss, 172.8ms
image 7/12 /content/input/BloodImage_00010.jpg: 480x640 20 RBCs, 2 WBCs, 173.7ms
image 8/12 /content/input/BloodImage_00011.jpg: 480x640 27 RBCs, 1 WBC, 1 Platelets, 161.0ms
image 9/12 /content/input/BloodImage_00012.jpg: 480x640 25 RBCs, 1 WBC, 2 Plateletss, 173.0ms
image 10/12 /content/input/BloodImage_00013.jpg: 480x640 26 RBCs, 1 WBC, 191.2ms
image 11/12 /content/input/BloodImage_00014.jpg: 480x640 23 RBCs, 1 WBC, 2 Plateletss, 161.5ms
image 12/12 /content

In [ ]:
import os
import glob
import shutil

# Define the output directory for the generated YOLO label files
output_labels_dir = '/content/output_part1/labeles'
os.makedirs(output_labels_dir, exist_ok=True)

# Define the output directory for the labeled images
output_labeled_images_dir = '/content/output_part1/labeled_images'
os.makedirs(output_labeled_images_dir, exist_ok=True)

print(f"Generating YOLO label files in: {output_labels_dir}")
print(f"Preparing to save labeled images in: {output_labeled_images_dir}")

# The 'results' variable holds the output from the last model.predict() call
if 'results' not in locals():
    print("Error: 'results' variable not found. Please run the model.predict() cell first.")
else:
    # Find the most recent prediction run directory for the labeled images
    yolo_default_output_base_dir = 'runs/detect'
    list_of_runs = glob.glob(os.path.join(yolo_default_output_base_dir, 'predict*'))
    latest_run_dir = None
    if list_of_runs:
        latest_run_dir = max(list_of_runs, key=os.path.getctime)
        print(f"Found latest YOLO prediction output at: {latest_run_dir}")
    else:
        print(f"Warning: No YOLO prediction run directories found in {yolo_default_output_base_dir}. Labeled images might not be available.")

    for result in results:
        # Get the original image filename
        image_filename = os.path.basename(result.path)
        base_name = os.path.splitext(image_filename)[0]

        # --- Generate and save YOLO label file ---
        label_file_path = os.path.join(output_labels_dir, f"{base_name}.txt")

        yolo_lines = []
        if result.boxes is not None:
            for box in result.boxes:
                class_id = int(box.cls[0])
                x_center_norm, y_center_norm, width_norm, height_norm = box.xywhn[0].tolist()
                yolo_lines.append(f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}")

        with open(label_file_path, 'w') as f:
            f.write('\n'.join(yolo_lines))
        print(f"Generated label for {image_filename} with {len(yolo_lines)} detections.")

        # --- Copy labeled image ---
        if latest_run_dir:
            # The labeled images have the same filename as the input images
            src_labeled_img_path = os.path.join(latest_run_dir, image_filename)
            dst_labeled_img_path = os.path.join(output_labeled_images_dir, image_filename)
            if os.path.exists(src_labeled_img_path):
                shutil.copy(src_labeled_img_path, dst_labeled_img_path)
                print(f"Copied labeled image {image_filename} to {output_labeled_images_dir}/")
            else:
                print(f"Warning: Labeled image {image_filename} not found in {latest_run_dir}")

    print("\nAll YOLO label files and labeled images generated/copied successfully!")


Generating YOLO label files in: /content/output_part1/labeles
Preparing to save labeled images in: /content/output_part1/labeled_images
Found latest YOLO prediction output at: runs/detect/predict
Generated label for BloodImage_00004.jpg with 19 detections.
Copied labeled image BloodImage_00004.jpg to /content/output_part1/labeled_images/
Generated label for BloodImage_00005.jpg with 28 detections.
Copied labeled image BloodImage_00005.jpg to /content/output_part1/labeled_images/
Generated label for BloodImage_00006.jpg with 24 detections.
Copied labeled image BloodImage_00006.jpg to /content/output_part1/labeled_images/
Generated label for BloodImage_00007.jpg with 28 detections.
Copied labeled image BloodImage_00007.jpg to /content/output_part1/labeled_images/
Generated label for BloodImage_00008.jpg with 24 detections.
Copied labeled image BloodImage_00008.jpg to /content/output_part1/labeled_images/
Generated label for BloodImage_00009.jpg with 27 detections.
Copied labeled image Bl

## WBD Extraction:

In [ ]:
import os
import cv2
from pathlib import Path


def extract_category_crops(images_dir, labels_dir, output_dir, target_category, class_names=None):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Get all label files
    label_files = sorted(Path(labels_dir).glob("*.txt"))

    crop_count = 0

    for label_file in label_files:
        # Get corresponding image file
        image_name = label_file.stem

        # Try common image extensions
        image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
            potential_path = Path(images_dir) / f"{image_name}{ext}"
            if potential_path.exists():
                image_path = potential_path
                break

        if image_path is None:
            print(f"Warning: Image not found for {image_name}")
            continue

        # Read image
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Warning: Could not read image {image_path}")
            continue

        height, width = image.shape[:2]

        # Read annotations
        with open(label_file, 'r') as f:
            lines = f.readlines()

        # Process each annotation
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            class_id = int(parts[0])

            # Check if this is the target category
            if class_id == target_category:
                #class_id x_center y_center width height (normalized 0-1)
                x_center = float(parts[1])
                y_center = float(parts[2])
                bbox_width = float(parts[3])
                bbox_height = float(parts[4])

                # Convert to pixel coordinates
                x_center_px = x_center * width
                y_center_px = y_center * height
                bbox_width_px = bbox_width * width
                bbox_height_px = bbox_height * height

                # Calculate bounding box corners
                x1 = int(x_center_px - bbox_width_px / 2)
                y1 = int(y_center_px - bbox_height_px / 2)
                x2 = int(x_center_px + bbox_width_px / 2)
                y2 = int(y_center_px + bbox_height_px / 2)

                # Ensure coordinates are within image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(width, x2)
                y2 = min(height, y2)

                # Crop image
                cropped = image[y1:y2, x1:x2]

                if cropped.size == 0:
                    continue

                # Generate output filename
                if class_names and target_category < len(class_names):
                    class_name = class_names[target_category]
                else:
                    class_name = f"class_{target_category}"

                output_filename = f"{class_name}_{image_name}_{idx}.jpg"
                output_path = os.path.join(output_dir, output_filename)

                # Save cropped image
                cv2.imwrite(output_path, cropped)
                crop_count += 1

In [ ]:
# Define the directories based on your request
images_dir = "./input"
labels_dir = "/content/output_part1/labeles" # Using the path from the previous output
output_dir = "/content/extracted_cells/WBC"

class_names = model.names

# Find the target category ID for 'WBC'
# This assumes 'WBC' is present in model.names
if 'WBC' in class_names:
    target_category_id = class_names.index('WBC')
    print(f"Found 'WBC' as category ID: {target_category_id}")
else:
    # Fallback to hardcoded ID if 'WBC' not found or class_names is not set up correctly
    target_category_id = 1
    print(f"Warning: 'WBC' not found in model.names. Using default category ID: {target_category_id}")

print(f"\nExtracting WBCs from '{images_dir}' using labels from '{labels_dir}'...")

extract_category_crops(
    images_dir=images_dir,
    labels_dir=labels_dir,
    output_dir=output_dir,
    target_category=target_category_id,
    class_names=class_names
)

print(f"\nWBC crops saved to: {output_dir}")


Extracting WBCs from './input' using labels from '/content/output_part1/labeles'...

WBC crops saved to: /content/extracted_cells/WBC


## Image Downsamplin and letterbox (for classification model):

In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import time

def rescale_images_for_yolo(input_dir, output_dir, target_size=(84, 84),
                            interpolation=cv2.INTER_AREA):

    os.makedirs(output_dir, exist_ok=True)

    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_files.extend(Path(input_dir).glob(ext))

    total_time = 0

    for img_path in image_files:
        start = time.time()

        # Read image
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Warning: Could not read {img_path}")
            continue

        resized = letterbox_resize(img, target_size, interpolation)

        # Save
        output_path = os.path.join(output_dir, img_path.name)
        cv2.imwrite(output_path, resized)

        total_time += time.time() - start

    print(f"Processed {len(image_files)} images in {total_time:.2f}s")
    print(f"Average time per image: {total_time/len(image_files)*1000:.2f}ms")


def letterbox_resize(img, target_size, interpolation=cv2.INTER_AREA):
    target_w, target_h = target_size
    h, w = img.shape[:2]

    # Calculate scaling factor
    scale = min(target_w / w, target_h / h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    # Resize image
    resized = cv2.resize(img, (new_w, new_h), interpolation=interpolation)

    # Create canvas with padding
    canvas = np.full((target_h, target_w, 3), (202, 207, 206), dtype=np.uint8)

    # Calculate padding
    pad_w = (target_w - new_w) // 2
    pad_h = (target_h - new_h) // 2

    # Place resized image on canvas
    canvas[pad_h:pad_h + new_h, pad_w:pad_w + new_w] = resized

    return canvas


rescaled_images = "/content/extracted_cells/WBC"
output_dir = "/content/input_model2"

rescale_images_for_yolo(
        input_dir=rescaled_images,
        output_dir=output_dir,
        target_size=(84, 84),
        interpolation=cv2.INTER_AREA
    )

Processed 13 images in 0.03s
Average time per image: 1.94ms


## WBC Classification:

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import glob

# 1. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Model Definition
num_classes = 5

model = models.resnet50(weights=None)
# Modify the final classification layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# 3. Load Weights
weights_path = "/content/best_wbc_resnet50.pth"
if os.path.exists(weights_path):
    # Load state_dict onto the appropriate device
    model.load_state_dict(torch.load(weights_path, map_location=device))
    print(f"Successfully loaded weights from {weights_path}")
else:
    print(f"Error: Weights file not found at {weights_path}. Please ensure it's in the /content/ folder.")
    print("Cannot proceed with classification without model weights.")

model = model.to(device)
model.eval() # Set model to evaluation mode

# 4. Image Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
])

# 5. Dataset and DataLoader
input_dir_for_classification = "/content/input_model2"

# Check if the directory exists and contains images
if not os.path.exists(input_dir_for_classification) or not os.listdir(input_dir_for_classification):
    print(f"Error: No images found in {input_dir_for_classification}. Please ensure previous steps ran correctly.")
else:
    print(f"Found images in {input_dir_for_classification} for classification.")

    # Create a list of image paths
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_paths.extend(glob.glob(os.path.join(input_dir_for_classification, ext)))
    image_paths.sort() # Ensure consistent order

    if not image_paths:
        print(f"No image files found in {input_dir_for_classification} with common extensions. Nothing to classify.")
    else:
        # Define class names based on the order your model was trained
        class_names = ["basophil", "eosinophil", "lymphocyte", "monocyte", "neutrophil"] # Updated class names

        print(f"\n--- Classification Results ---")
        with torch.no_grad(): # Disable gradient calculation for inference
            for img_path in image_paths:
                img_name = os.path.basename(img_path)
                try:
                    image = Image.open(img_path).convert("RGB") # Ensure 3 channels for ResNet
                    input_tensor = transform(image)
                    input_batch = input_tensor.unsqueeze(0).to(device) # Add a batch dimension

                    output = model(input_batch)
                    probabilities = torch.nn.functional.softmax(output, dim=1)
                    # Get the predicted class with the highest probability
                    _, predicted_class_idx = torch.max(probabilities, 1)
                    confidence = probabilities[0][predicted_class_idx].item()

                    if predicted_class_idx.item() < len(class_names):
                        predicted_class_name = class_names[predicted_class_idx.item()]
                    else:
                        predicted_class_name = f"Class_{predicted_class_idx.item()}" # Fallback if index out of bounds

                    print(f"Image: {img_name} -> Predicted Class: {predicted_class_name} (Confidence: {confidence:.2f})")

                except Exception as e:
                    print(f"Could not process image {img_name}: {e}")

        print("\nClassification complete!")


Using device: cpu
Successfully loaded weights from /content/best_wbc_resnet50.pth
Found images in /content/input_model2 for classification.

--- Classification Results ---
Image: WBC_BloodImage_00004_2.jpg -> Predicted Class: eosinophil (Confidence: 0.53)
Image: WBC_BloodImage_00005_5.jpg -> Predicted Class: neutrophil (Confidence: 0.87)
Image: WBC_BloodImage_00006_2.jpg -> Predicted Class: neutrophil (Confidence: 0.96)
Image: WBC_BloodImage_00007_2.jpg -> Predicted Class: eosinophil (Confidence: 0.40)
Image: WBC_BloodImage_00008_1.jpg -> Predicted Class: monocyte (Confidence: 0.73)
Image: WBC_BloodImage_00009_2.jpg -> Predicted Class: eosinophil (Confidence: 0.98)
Image: WBC_BloodImage_00010_5.jpg -> Predicted Class: monocyte (Confidence: 0.80)
Image: WBC_BloodImage_00010_7.jpg -> Predicted Class: neutrophil (Confidence: 0.81)
Image: WBC_BloodImage_00011_1.jpg -> Predicted Class: eosinophil (Confidence: 0.50)
Image: WBC_BloodImage_00012_1.jpg -> Predicted Class: eosinophil (Confidence

## Combine and Display Comprehensive Output

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import glob

wbc_classification_results = [] # Initialize the list to store results

# 1. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Model Definition
num_classes = 5

model = models.resnet50(weights=None)
# Modify the final classification layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# 3. Load Weights
weights_path = "/content/best_wbc_resnet50.pth"
if os.path.exists(weights_path):
    # Load state_dict onto the appropriate device
    model.load_state_dict(torch.load(weights_path, map_location=device))
    print(f"Successfully loaded weights from {weights_path}")
else:
    print(f"Error: Weights file not found at {weights_path}. Please ensure it's in the /content/ folder.")
    print("Cannot proceed with classification without model weights.")

model = model.to(device)
model.eval() # Set model to evaluation mode

# 4. Image Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
])

# 5. Dataset and DataLoader
input_dir_for_classification = "/content/input_model2"

# Check if the directory exists and contains images
if not os.path.exists(input_dir_for_classification) or not os.listdir(input_dir_for_classification):
    print(f"Error: No images found in {input_dir_for_classification}. Please ensure previous steps ran correctly.")
else:
    print(f"Found images in {input_dir_for_classification} for classification.")

    # Create a list of image paths
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_paths.extend(glob.glob(os.path.join(input_dir_for_classification, ext)))
    image_paths.sort() # Ensure consistent order

    if not image_paths:
        print(f"No image files found in {input_dir_for_classification} with common extensions. Nothing to classify.")
    else:
        # Define class names based on the order your model was trained
        class_names = ["basophil", "eosinophil", "lymphocyte", "monocyte", "neutrophil"]

        print(f"\n--- Classification Results ---")
        with torch.no_grad(): # Disable gradient calculation for inference
            for img_path in image_paths:
                img_name = os.path.basename(img_path)
                try:
                    image = Image.open(img_path).convert("RGB") # Ensure 3 channels for ResNet
                    input_tensor = transform(image)
                    input_batch = input_tensor.unsqueeze(0).to(device) # Add a batch dimension

                    output = model(input_batch)
                    probabilities = torch.nn.functional.softmax(output, dim=1)
                    # Get the predicted class with the highest probability
                    _, predicted_class_idx = torch.max(probabilities, 1)
                    confidence = probabilities[0][predicted_class_idx].item()

                    if predicted_class_idx.item() < len(class_names):
                        predicted_class_name = class_names[predicted_class_idx.item()]
                    else:
                        predicted_class_name = f"Class_{predicted_class_idx.item()}" # Fallback if index out of bounds

                    # Extract original_image_source (e.g., 'WBC_BloodImage_00016_1.jpg' -> 'BloodImage_00016.jpg')
                    parts = img_name.split('_')
                    # Rejoin all parts except the first ('WBC') and the last (index.jpg) then add '.jpg'
                    original_image_source = '_'.join(parts[1:-1]) + '.jpg'

                    wbc_classification_results.append({
                        'crop_path': img_path,
                        'predicted_class': predicted_class_name,
                        'confidence': confidence,
                        'original_image_source': original_image_source
                    })

                    print(f"Image: {img_name} -> Predicted Class: {predicted_class_name} (Confidence: {confidence:.2f})")

                except Exception as e:
                    print(f"Could not process image {img_name}: {e}")

        print("\nClassification complete!")

Using device: cpu
Successfully loaded weights from /content/best_wbc_resnet50.pth
Found images in /content/input_model2 for classification.

--- Classification Results ---
Image: WBC_BloodImage_00004_2.jpg -> Predicted Class: eosinophil (Confidence: 0.53)
Image: WBC_BloodImage_00005_5.jpg -> Predicted Class: neutrophil (Confidence: 0.87)
Image: WBC_BloodImage_00006_2.jpg -> Predicted Class: neutrophil (Confidence: 0.96)
Image: WBC_BloodImage_00007_2.jpg -> Predicted Class: eosinophil (Confidence: 0.40)
Image: WBC_BloodImage_00008_1.jpg -> Predicted Class: monocyte (Confidence: 0.73)
Image: WBC_BloodImage_00009_2.jpg -> Predicted Class: eosinophil (Confidence: 0.98)
Image: WBC_BloodImage_00010_5.jpg -> Predicted Class: monocyte (Confidence: 0.80)
Image: WBC_BloodImage_00010_7.jpg -> Predicted Class: neutrophil (Confidence: 0.81)
Image: WBC_BloodImage_00011_1.jpg -> Predicted Class: eosinophil (Confidence: 0.50)
Image: WBC_BloodImage_00012_1.jpg -> Predicted Class: eosinophil (Confidence

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import glob
import pandas as pd
import numpy as np
import math
import io

# --- 1. Define input and output directories and class names ---
input_images_dir = './input'
output_labeled_images_dir = '/content/output_part1/labeled_images'
labels_dir = '/content/output_part1/labeles'

# Initialize class names for detection model (RBC, WBC, Platelets)
class_names_from_model = ['RBC', 'WBC', 'Platelets']
# Attempt to get them from 'model' if it's available in the kernel state
if 'model' in locals() and hasattr(model, 'names'):
    class_names_from_model = model.names
class_id_to_name = {i: name for i, name in enumerate(class_names_from_model)}

# Initialize class names for WBC classification model
wbc_classification_class_names = ["basophil", "eosinophil", "lymphocyte", "monocyte", "neutrophil"] # Updated class names

if 'class_names' in locals() and isinstance(class_names, list) and len(class_names) == 5:
    wbc_classification_class_names = class_names

# --- 2. Ensure wbc_classification_results global list is available ---
# This list should now be populated from the previous step
if 'wbc_classification_results' not in locals() or not isinstance(wbc_classification_results, list):
    print("Warning: 'wbc_classification_results' not found or not a list. Initializing as empty.")
    wbc_classification_results = []

# --- Helper function to render DataFrame as an image ---
def dataframe_to_image(df, title="", dpi=100):
    fig, ax = plt.subplots(figsize=(df.shape[1] * 1.5, df.shape[0] * 0.6), dpi=dpi)
    ax.axis('off')
    ax.axis('tight')
    # Create the table
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    ax.set_title(title, fontsize=12, pad=10)

    # Save to buffer and load as PIL Image
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', dpi=dpi)
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    return img

# --- Helper function to render WBC crops grid as an image ---
def wbc_crops_grid_to_image(wbc_crops_data, original_img_filename, dpi=100):
    if not wbc_crops_data:
        # Create a placeholder image if no WBCs found for this image
        dummy_img = Image.new('RGB', (300, 150), color = (200, 200, 200))
        draw = ImageDraw.Draw(dummy_img)
        text = f"No WBCs found\nfor {original_img_filename}"
        # Simple text positioning - ideally dynamically calculate for centering
        draw.text((10, 50), text, fill=(0,0,0))
        return dummy_img

    num_wbc_crops = len(wbc_crops_data)
    num_cols = 3
    num_rows = math.ceil(num_wbc_crops / num_cols)

    # Calculate figure size dynamically
    fig_width = num_cols * 2.5 # Each crop about 2.5 inches wide
    fig_height = num_rows * 2.5 + 0.5 # Each crop about 2.5 inches high + some title space
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(fig_width, fig_height), dpi=dpi)

    if num_rows == 1 and num_cols == 1 and num_wbc_crops == 1: # Handle single subplot case
        axes = np.array([axes]) # Make it iterable like other cases
    else:
        axes = axes.flatten()

    for i, wbc_result in enumerate(wbc_crops_data):
        crop_path = wbc_result['crop_path']
        predicted_class = wbc_result['predicted_class']
        confidence = wbc_result['confidence']

        if i < len(axes):
            ax = axes[i]
            try:
                crop_img = Image.open(crop_path)
                ax.imshow(crop_img)
                ax.set_title(f"{predicted_class.capitalize()}\n(Conf: {confidence:.2f})", fontsize=8)
                ax.axis('off')
            except Exception as e:
                print(f"Error displaying crop {os.path.basename(crop_path)}: {e}")
                ax.text(0.5, 0.5, 'Error', ha='center', va='center', transform=ax.transAxes, color='red')
                ax.axis('off')

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f"Classified WBCs from {original_img_filename}", fontsize=14, y=0.99)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap

    # Save to buffer and load as PIL Image
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', dpi=dpi)
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    return img

# --- 3. Get a sorted list of all original image filenames ---
original_image_filenames = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
    original_image_filenames.extend([os.path.basename(p) for p in glob.glob(os.path.join(input_images_dir, ext))])
original_image_filenames.sort()

if not original_image_filenames:
    print(f"No original images found in {input_images_dir} to process.")
else:
    print(f"Processing {len(original_image_filenames)} original images...")

    for original_img_filename in original_image_filenames:
        print(f"\n--- Processing {original_img_filename} ---")

        # a. Get the Labeled Image
        labeled_img_path = os.path.join(output_labeled_images_dir, original_img_filename)
        if os.path.exists(labeled_img_path):
            labeled_img_pil = Image.open(labeled_img_path).convert('RGB')
        else:
            print(f"Warning: Labeled image not found for {original_img_filename}. Creating placeholder.")
            labeled_img_pil = Image.new('RGB', (640, 480), color = (100, 100, 100))
            draw = ImageDraw.Draw(labeled_img_pil)
            draw.text((10, 10), f"Labeled Image Missing:\n{original_img_filename}", fill=(255,255,255))

        # b. Create a table image summarizing blood cell counts
        base_name = os.path.splitext(original_img_filename)[0]
        label_file_path = os.path.join(labels_dir, f"{base_name}.txt")

        counts_data = {'Image': original_img_filename, 'RBC': 0, 'WBC': 0, 'Platelets': 0}

        if os.path.exists(label_file_path):
            with open(label_file_path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    class_id = int(parts[0])
                    class_name = class_id_to_name.get(class_id, f"Unknown_{class_id}")
                    if class_name in counts_data: # Only count known classes
                        counts_data[class_name] += 1
        else:
            print(f"Warning: Label file not found for {original_img_filename}. Counts will be zero.")

        counts_df = pd.DataFrame([counts_data])
        table_img_pil = dataframe_to_image(counts_df, title="Cell Detection Counts")

        # c. Render a 3-column grid of its classified WBC crops
        wbc_crops_for_image = [
            res for res in wbc_classification_results
            if res['original_image_source'] == original_img_filename
        ]
        wbc_grid_img_pil = wbc_crops_grid_to_image(wbc_crops_for_image, original_img_filename)

        # d. Combine these three visual elements into a single, cohesive image

        # Resize labeled image to a reasonable width if too large, maintain aspect ratio
        target_labeled_width = 600
        if labeled_img_pil.width > target_labeled_width:
            labeled_img_pil = labeled_img_pil.resize((target_labeled_width, int(labeled_img_pil.height * target_labeled_width / labeled_img_pil.width)), Image.Resampling.LANCZOS)

        # Find the maximum width among the three for consistent display
        max_combine_width = max(labeled_img_pil.width, table_img_pil.width, wbc_grid_img_pil.width)

        # Create new images with the common width, padding with white if narrower
        def pad_image_to_width(img, target_width, fill_color=(255,255,255)):
            if img.width < target_width:
                new_img = Image.new('RGB', (target_width, img.height), fill_color)
                offset_x = (target_width - img.width) // 2
                new_img.paste(img, (offset_x, 0))
                return new_img
            return img

        padded_labeled_img = pad_image_to_width(labeled_img_pil, max_combine_width)
        padded_table_img = pad_image_to_width(table_img_pil, max_combine_width)
        padded_wbc_grid_img = pad_image_to_width(wbc_grid_img_pil, max_combine_width)

        final_height = padded_labeled_img.height + padded_table_img.height + padded_wbc_grid_img.height
        combined_image = Image.new('RGB', (max_combine_width, final_height), color=(255, 255, 255))

        y_offset = 0
        combined_image.paste(padded_labeled_img, (0, y_offset))
        y_offset += padded_labeled_img.height
        combined_image.paste(padded_table_img, (0, y_offset))
        y_offset += padded_table_img.height
        combined_image.paste(padded_wbc_grid_img, (0, y_offset))

        # Display the combined image
        plt.figure(figsize=(max_combine_width / 100, final_height / 100)) # Adjust figsize based on image px and dpi
        plt.imshow(combined_image)
        plt.title(f"Comprehensive Analysis for: {original_img_filename}")
        plt.axis('off')
        plt.show()

print("\nFinished generating and displaying comprehensive outputs for all images.")

Output hidden; open in https://colab.research.google.com to view.